<a href="https://colab.research.google.com/github/Khoan-IT/TrainingWithTPU/blob/main/ResNet50_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from time import process_time, time
from keras.applications.resnet50 import ResNet50
from keras.datasets import cifar100
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [42]:
VALIDATION_SPLIT = 0.2
TARGET_SIZE = [32,32]
BATCH_SIZE = 512

(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
data_size = len(x_train)
temp = []
for i in range(data_size):
  one_hot = np.eye(1,100,int(y_train[i]),dtype = np.int64)
  temp.append(one_hot.tolist()[0])
y_train = temp
x_train, x_test = x_train / 255, x_test / 255 # normalize data
x_val = x_train[-int(VALIDATION_SPLIT*len(y_train)):]
y_val = y_train[-int(VALIDATION_SPLIT*len(y_train)):]
x_train = x_train[:int(len(y_train)-VALIDATION_SPLIT*len(y_train))]
y_train = y_train[:int(len(y_train)-VALIDATION_SPLIT*len(y_train))]



In [43]:
AUTOTUNE = tf.data.experimental.AUTOTUNE # https://www.tensorflow.org/guide/data_performance

def _input_fn(x,y, BATCH_SIZE,train=False):
  ds = tf.data.Dataset.from_tensor_slices((x,y))
  ds = ds.shuffle(buffer_size=data_size)
  if train:
    ds = ds.repeat()
  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds =_input_fn(x_train,y_train, BATCH_SIZE,train=True) #PrefetchDataset object
validation_ds =_input_fn(x_val,y_val, BATCH_SIZE, train = False) #PrefetchDataset object
test_ds =_input_fn(x_test,y_test, BATCH_SIZE) #PrefetchDataset object

In [44]:
model = tf.keras.applications.ResNet50(weights=None, input_shape=[*TARGET_SIZE, 3], classes=100)
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'])

model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [45]:
EPOCHS = 50
STEPS_PER_EPOCH = len(x_train)//BATCH_SIZE
VALIDATION_STEPS = len(x_val)//BATCH_SIZE

In [46]:
t1_start = time() 
his = model.fit(train_ds,
    epochs = EPOCHS,
    validation_data = validation_ds,
    validation_steps = VALIDATION_STEPS,
    steps_per_epoch = STEPS_PER_EPOCH,
    verbose=True
)

t1_stop = time()    
print("Elapsed time during the whole program in seconds model", t1_stop-t1_start) 

Epoch 1/50
78/78 [==============================] - 24s 225ms/step - loss: 5.1015 - accuracy: 0.0393 - val_loss: 4.8277 - val_accuracy: 0.0105
Epoch 2/50
78/78 [==============================] - 17s 219ms/step - loss: 3.6037 - accuracy: 0.1585 - val_loss: 5.4062 - val_accuracy: 0.0117
Epoch 3/50
78/78 [==============================] - 17s 218ms/step - loss: 3.3130 - accuracy: 0.2179 - val_loss: 5.2957 - val_accuracy: 0.0122
Epoch 4/50
78/78 [==============================] - 17s 215ms/step - loss: 2.9676 - accuracy: 0.2805 - val_loss: 4.8852 - val_accuracy: 0.0378
Epoch 5/50
78/78 [==============================] - 17s 213ms/step - loss: 2.6332 - accuracy: 0.3424 - val_loss: 6.9429 - val_accuracy: 0.0162
Epoch 6/50
78/78 [==============================] - 17s 214ms/step - loss: 2.7324 - accuracy: 0.3315 - val_loss: 455.5096 - val_accuracy: 0.0065
Epoch 7/50
78/78 [==============================] - 17s 216ms/step - loss: 3.0629 - accuracy: 0.2734 - val_loss: 5501.2305 - val_accuracy: 0